# Automated Code Smell Detection Using Large Language Models

## Problem Statement

Code smells - patterns in source code that indicate potential design problems - are critical indicators of software quality issues. Traditionally, identifying code smells has relied on:

1. Manual code reviews (time-consuming and subjective)
2. Static analysis tools (limited to predefined patterns)
3. Software metrics (often producing false positives)

These approaches either require significant human effort or lack the contextual understanding needed to identify subtle design issues.

## How Generative AI Solves This Problem

This notebook demonstrates how Large Language Models (LLMs) can transform code smell detection by:

- **Contextual understanding**: Analyzing code with deep semantic understanding rather than just pattern matching
- **Knowledge integration**: Leveraging structured knowledge about different code smell types and refactoring solutions
- **Natural language reasoning**: Providing detailed explanations and actionable recommendations in human-readable format

## What This Notebook Demonstrates

This end-to-end implementation shows how to:

1. Build a knowledge base of code smells using vector embeddings and DeepLake
2. Create a retrieval-augmented generation (RAG) system to provide context about code smells
3. Analyze Java code files to detect multiple types of code smells
4. Generate structured output with smell locations, severity, and refactoring suggestions
5. Evaluate detection quality against manually labeled data

The approach provides a practical showcase of how generative AI can be applied to enhance software development practices through automated code quality assessment.

**Now, let's begin.**


*install all needed dependencies:*

In [1]:
%pip install -q -U "langchain-google-genai" "deeplake" "langchain" "langchain-text-splitters" "langchain-community" "tiktoken" "google-ai-generativelanguage==0.6.15" "deeplake[enterprise]<4.0.0"
%pip install pillow lz4 python-dotenv sonar-tools mysql-connector-python pandas


[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
  Using cached argparse-1.4.0-py2.py3-none-any.whl.metadata (2.8 kB)
Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)

[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


now initialize deep lake vectore store and store structured info about code smells in it

we will use 2 repos as a data sources:

* pixel-dungeon - as an example of software that contains numerous code smells.
* smells - a comprehensive classification of code smells. we will use structured information about code smells from this repository in our DeepLake database.

In [ ]:
!git clone https://github.com/watabou/pixel-dungeon.git
!git clone https://github.com/Luzkan/smells.git
!git clone https://github.com/alibaba/arthas.git
!ls
!cd ./arthas && git checkout 4fc682265ce9e8db0101f978d32b142af6751493


fatal: destination path 'pixel-dungeon' already exists and is not an empty directory.
fatal: destination path 'smells' already exists and is not an empty directory.
fatal: destination path 'arthas' already exists and is not an empty directory.
arthas             import pytest.json smells
experiment1.ipynb  pixel-dungeon
HEAD is now at 4fc68226 classloader command support jdk.internal.loader.ClassLoaders$AppClassLoade. #2350


get all files with structured info about code smells

In [2]:
from glob import glob
from IPython.display import Markdown, display

from langchain.vectorstores import DeepLake
from langchain.document_loaders import TextLoader
from langchain_text_splitters import (
    Language,
    RecursiveCharacterTextSplitter,
)
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain.chains import RetrievalQA
import deeplake

/Users/havriil.pietukhin/uni/masterThesis/code/smellai/.venv/lib/python3.13/site-packages/deeplake/util/check_latest_version.py:32: UserWarning: A newer version of deeplake (4.2.4) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(
/Users/havriil.pietukhin/uni/masterThesis/code/smellai/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Set up your API key

To run the following cell, your API key must be stored it in a Kaggle secret named GOOGLE_API_KEY.

If you don't already have an API key, you can grab one from AI Studio. You can find detailed instructions in the docs.

To make the key available through Kaggle secrets, choose Secrets from the Add-ons menu and follow the instructions to add your key or enable it for this notebook.

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

api_key = os.environ["GOOGLE_API_KEY"]

In [7]:
#smells_match = "smells/content/smells/**/*.md"
smell_files = [
    "smells/content/smells/duplicated-code.md",
    "smells/content/smells/long-method.md", 
    "smells/content/smells/large-class.md",
    "smells/content/smells/long-parameter-list.md"
]

# Use glob to find each file individually
all_smells = []
for file_pattern in smell_files:
    matches = glob(file_pattern)
    all_smells.extend(matches)

# Print the first 10 files
print(f"Smells implemented in Sonarqube are represented by {len(all_smells)} markdown files:")
for file in all_smells[:5]:
    print(f"  {file}")

Smells implemented in Sonarqube are represented by 4 markdown files:
  smells/content/smells/duplicated-code.md
  smells/content/smells/long-method.md
  smells/content/smells/large-class.md
  smells/content/smells/long-parameter-list.md


Each file with a matching path will be loaded and split by RecursiveCharacterTextSplitter. only Markdown files with structured content will be processed

In [ ]:
# common seperators used for Python files
RecursiveCharacterTextSplitter.get_separators_for_language(Language.MARKDOWN)

['\n#{1,6} ',
 '```\n',
 '\n\\*\\*\\*+\n',
 '\n---+\n',
 '\n___+\n',
 '\n\n',
 '\n',
 ' ',
 '']

In [ ]:
%pip install python-frontmatter
%pip install -qU pypdf

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from langchain_text_splitters import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.schema import Document
import os

# Define headers that match your code smell documentation structure
headers_to_split_on = [
    ("#", "Title"),
    ("##", "Section"),
    ("###", "Subsection")
]

docs = []
for file in all_smells:
    try:
        # First load the file as text
        with open(file, 'r', encoding='utf-8') as f:
            content = f.read()
        
        # Extract the main content (everything after the frontmatter)
        if '---' in content:
            # Find the second occurrence of '---' which ends the frontmatter
            parts = content.split('---', 2)
            if len(parts) >= 3:
                markdown_content = '---' + parts[2]  # Keep the separator for proper markdown parsing
            else:
                markdown_content = content
        else:
            markdown_content = content
            
        # Split by markdown headers
        markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
        header_splits = markdown_splitter.split_text(markdown_content)
        
        # Add source file info to metadata
        for doc in header_splits:
            doc.metadata['source'] = file
        
        # Add to collection
        docs.extend(header_splits)
        
    except Exception as e:
        print(f"Error processing {file}: {e}")

print(f"Created {len(docs)} chunks from {len(all_smells)} files")

Created 324 chunks from 56 files


In [ ]:
# define path to database
dataset_path = 'mem://deeplake/smells'

In [ ]:
# define the embedding model
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

In [ ]:
smell_db = DeepLake.from_documents(docs, embeddings, dataset_path=dataset_path)

Creating 324 embeddings in 1 batches of size 324:: 100%|██████████| 1/1 [00:05<00:00,  5.46s/it]

Dataset(path='mem://deeplake/smells', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype      shape      dtype  compression
  -------    -------    -------    -------  ------- 
   text       text      (324, 1)     str     None   
 metadata     json      (324, 1)     str     None   
 embedding  embedding  (324, 768)  float32   None   
    id        text      (324, 1)     str     None   


In [ ]:
retriever = smell_db.as_retriever()
retriever.search_kwargs['distance_metric'] = 'cos'
retriever.search_kwargs['k'] = 20 # number of documents to return

In [ ]:
# define the chat model
llm = ChatGoogleGenerativeAI(model = "gemini-2.0-flash")

In [ ]:
qa = RetrievalQA.from_llm(llm, retriever=retriever)

In [ ]:
# a helper function for calling retrival chain
def call_qa_chain(prompt):
  response = qa.invoke(prompt)
  display(Markdown(response["result"]))

In [ ]:
call_qa_chain("List a recommendations on how to get rid of 'God class' smell, rating from more simple to more sophisticated strategies.")

Here are some refactoring strategies to address the 'God Class' smell, ranging from simpler to more sophisticated:

1.  **Extract Class:** ([Origin] - Martin Fowler, \_"Refactoring: Improving the Design of Existing Code"\_ (1999)) This involves identifying related responsibilities within the God Class and moving them to a new, separate class.
2.  **Extract Function:** ([Origin] - Martin Fowler, \_"Refactoring: Improving the Design of Existing Code"\_ (1999))  If a part of the God Class's logic can be isolated into a well-defined function, extract it.
3.  **Move Function:** ([Origin] - Martin Fowler, \_"Refactoring: Improving the Design of Existing Code (3rd Edition)"\_ (1999)) If a function seems more related to another class, move it to that class.
4.  **Replace Type Code/Conditional Logic with State/Strategy:** When a God Class uses conditional logic (e.g., `if/else` or `switch` statements) based on type codes, consider using the State or Strategy pattern to delegate behavior to separate classes.
5.  **Move Embellishment to Decorator:** ([Origin] - Martin Fowler, \_"Refactoring: Improving the Design of Existing Code"\_ (1999))

In [ ]:
import os
import enum
from glob import glob
from pathlib import Path

# Get all markdown files in the content/smells directory
smell_files = all_smells

# Create enum class dynamically
def create_code_smell_enum():
    # Process each filename to create enum-friendly names
    enum_entries = {}
    
    for file_path in smell_files:
        # Extract filename without extension
        filename = Path(file_path).stem
        
        # Convert kebab-case to UPPER_SNAKE_CASE for enum names
        enum_name = filename.replace('-', '_').upper()
        
        # Use the original filename (without extension) as the value
        enum_entries[enum_name] = filename
    
    # Create and return the Enum class
    return enum.Enum('CodeSmell', enum_entries)

# Create the enum
CodeSmell = create_code_smell_enum()

# Display the enum members
print(f"Created enum with {len(CodeSmell)} code smells:")
for smell in CodeSmell:
    print(f"{smell.name} = '{smell.value}'")

# Example usage
print("\nExample usage:")
print(f"CodeSmell.DEAD_CODE = '{CodeSmell.DEAD_CODE.value}'")
print(f"CodeSmell.FEATURE_ENVY = '{CodeSmell.FEATURE_ENVY.value}'")

# You can also look up an enum by value
def get_smell_by_name(name):
    for smell in CodeSmell:
        if smell.value == name:
            return smell
    return None

print("\nLooking up by name:")
print(f"get_smell_by_name('dead-code') = {get_smell_by_name('dead-code')}")

Created enum with 56 code smells:
MUTABLE_DATA = 'mutable-data'
INAPPROPRIATE_STATIC = 'inappropriate-static'
LONG_METHOD = 'long-method'
IMPERATIVE_LOOPS = 'imperative-loops'
REFUSED_BEQUEST = 'refused-bequest'
SIDE_EFFECTS = 'side-effects'
INDECENT_EXPOSURE = 'indecent-exposure'
CLEVER_CODE = 'clever-code'
DUPLICATED_CODE = 'duplicated-code'
UNCOMMUNICATIVE_NAME = 'uncommunicative-name'
SPECULATIVE_GENERALITY = 'speculative-generality'
FATE_OVER_ACTION = 'fate-over-action'
CALLBACK_HELL = 'callback-hell'
LONG_PARAMETER_LIST = 'long-parameter-list'
MAGIC_NUMBER = 'magic-number'
TEMPORARY_FIELD = 'temporary-field'
OBSCURED_INTENT = 'obscured-intent'
PARALLEL_INHERITANCE_HIERARCHIES = 'parallel-inheritance-hierarchies'
INCONSISTENT_NAMES = 'inconsistent-names'
COMPLICATED_REGEX_EXPRESSION = 'complicated-regex-expression'
ODDBALL_SOLUTION = 'oddball-solution'
HIDDEN_DEPENDENCIES = 'hidden-dependencies'
COMPLICATED_BOOLEAN_EXPRESSION = 'complicated-boolean-expression'
BASE_CLASS_DEPENDS_O

In [ ]:
%pip install langchain_community langchain-text-splitters pypdf

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from pydantic import BaseModel, Field
from typing import List, Optional, Literal
import enum

# Define your structured output schema using Pydantic
class CodeSmellSeverity(str, enum.Enum):
    HIGH = "HIGH"
    MEDIUM = "MEDIUM" 
    LOW = "LOW"

class CodeSmellDetection(BaseModel):
    smell_type: str = Field(description="The type of code smell detected")
    location: str = Field(description="Where in the code the smell was found")
    severity: CodeSmellSeverity = Field(description="How severe the smell is")
    description: str = Field(description="Brief explanation of the issue")
    refactoring_suggestion: str = Field(description="How to fix the code smell")
    code_example: Optional[str] = Field(None, description="Example code showing the fix")

class CodeAnalysisResult(BaseModel):
    analysis_summary: str = Field(description="Overall summary of code quality")
    smells_detected: List[CodeSmellDetection] = Field(description="List of detected code smells")

# Create a parser for the structured output
parser = PydanticOutputParser(pydantic_object=CodeAnalysisResult)

# Create a prompt template that includes formatting instructions
code_analysis_prompt = PromptTemplate(
    template="""
You are an expert code analyst. Analyze the following code for code smells:

```java
{code}
{format_instructions}

Only identify code smells from this list: {valid_smells} """, input_variables=["code", "valid_smells"], partial_variables={"format_instructions": parser.get_format_instructions()} )

In [ ]:
def analyze_code_with_structure(code_content):
    # Get list of valid smells for reference
    valid_smells = ", ".join([smell.name for smell in CodeSmell])
    
    # Create a prompt that sends the code directly to the QA system
    analysis_prompt = f"""
    Analyze the following code for code smells:
    
    ```java
    {code_content}
    ```
    
    What code smells can you identify in this code and why? Only consider these code smell types: {valid_smells}.
    
    For each smell found, provide:
    1. The exact smell type (from the list provided)
    2. Location in the code (file, line numbers, method names)
    3. Severity (HIGH, MEDIUM, LOW)
    4. Description of why this is a code smell
    5. Refactoring suggestion to fix the issue
    6. Optional: Example code showing the fix
    
    Format your response as a structured JSON object matching this schema:
    {parser.get_format_instructions()}
    """
    
    # Use the QA system which leverages the smell knowledge base
    try:
        qa_result = qa.invoke(analysis_prompt)
        
        # Try to extract and parse JSON from the result
        output_text = qa_result["result"]
        parsed_output = parser.parse(output_text)
        return parsed_output
    except Exception as e:
        print(f"Error in code smell analysis: {e}")
        print(f"Raw QA output: {qa_result['result'] if 'qa_result' in locals() else 'No output'}")
        
        # Fallback to direct LLM call if QA system parsing fails
        try:
            direct_output = llm.invoke(analysis_prompt)
            parsed_output = parser.parse(direct_output.content)
            return parsed_output
        except Exception as e2:
            print(f"Fallback also failed: {e2}")
            return None

In [ ]:
def display_code_analysis(file_path): # Read the file content 
    with open(file_path, 'r') as f: code_content = f.read()
        # Analyze the code
    analysis = analyze_code_with_structure(code_content)

    if not analysis:
        print("Failed to analyze code.")
        return

    # Display results
    print(f"Analysis Summary: {analysis.analysis_summary}\n")
    print(f"Detected {len(analysis.smells_detected)} code smells:")

    for i, smell in enumerate(analysis.smells_detected, 1):
        print(f"\n{i}. {smell.smell_type} ({smell.severity})")
        print(f"   Location: {smell.location}")
        print(f"   Description: {smell.description}")
        print(f"   Refactoring: {smell.refactoring_suggestion}")
        if smell.code_example:
            print(f"\n   Example fix:\n   ```\n{smell.code_example}\n   ```")

In [ ]:
# Analyze a Java file from the pixel-dungeon repository
java_file = "pixel-dungeon/src/com/watabou/pixeldungeon/levels/HallsLevel.java"
display_code_analysis(java_file)

Analysis Summary: The code exhibits several code smells related to magic numbers, duplicated code, obscured intent, and conditional complexity. These smells reduce readability, maintainability, and extensibility.

Detected 14 code smells:

1. MAGIC_NUMBER (CodeSmellSeverity.LOW)
   Location: HallsLevel.java:26
   Description: The numbers 25 and 1 are used without clear explanation of their meaning.
   Refactoring: Replace magic numbers with named constants to improve readability.

   Example fix:
   ```
private static final int BASE_VIEW_DISTANCE = 25;
private static final int MIN_VIEW_DISTANCE = 1;
viewDistance = Math.max( BASE_VIEW_DISTANCE - Dungeon.depth, MIN_VIEW_DISTANCE );
   ```

2. MAGIC_NUMBER (CodeSmellSeverity.LOW)
   Location: HallsLevel.java:27
   Description: The hex color codes 0x801500 and 0xa68521 are used without clear explanation of their meaning.
   Refactoring: Replace magic numbers with named constants to improve readability.

   Example fix:
   ```
private stati

In [ ]:
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain.schema import Document
from pydantic import BaseModel, Field
from typing import List, Optional, Dict, Any
import enum
import json
from IPython.display import Markdown, display

# 1. Define evaluation schema
class EvaluationScore(str, enum.Enum):
    EXCELLENT = "EXCELLENT"
    GOOD = "GOOD"
    ACCEPTABLE = "ACCEPTABLE"
    POOR = "POOR"
    INCORRECT = "INCORRECT"

class SmellEvaluation(BaseModel):
    detected_smell: str = Field(description="The detected code smell type")
    location: str = Field(description="Where the smell was detected")
    ground_truth_match: Optional[str] = Field(None, description="The matching ground truth smell if any")
    score: EvaluationScore = Field(description="Evaluation score for this detection")
    justification: str = Field(description="Explanation for the rating")

class CodeSmellEvaluationResult(BaseModel):
    overall_score: float = Field(description="Overall evaluation score out of 5")
    precision: float = Field(description="Ratio of correctly identified smells to all detections")
    recall: float = Field(description="Ratio of correctly identified smells to all actual smells")
    evaluations: List[SmellEvaluation] = Field(description="Individual smell evaluations")
    summary: str = Field(description="Summary of evaluation results")

# 2. Create the evaluation prompt template
eval_template = """
# Instruction
You are an expert evaluator specializing in code smell detection. Your task is to evaluate the quality of code smell detections by comparing them with ground truth data.

# Evaluation
## Metric Definition
You will be assessing code smell detection quality, which measures how accurately the system identifies:
1. The correct type of code smell
2. The correct location of the smell (file, line numbers, method)

## Criteria
Smell Type Accuracy: The detected smell type matches the actual smell type in the code.
Location Accuracy: The location identified for the smell (line numbers, method, class) matches where the smell actually exists.
Justification Quality: The explanation provided for the smell makes sense and correctly describes the issue.
Refactoring Relevance: The suggested refactoring is appropriate for the identified smell.

## Rating Rubric
EXCELLENT: Perfect match of smell type and exact location (score: 5).
GOOD: Correct smell type with minor location imprecision (score: 4).
ACCEPTABLE: Partial match (either correct smell type or approximate location) (score: 3).
POOR: Wrong smell type but area of concern correctly identified (score: 2).
INCORRECT: Completely incorrect detection (wrong smell type and location) (score: 1).

## Evaluation Steps
STEP 1: For each detected smell, find any matching ground truth smells.
STEP 2: Evaluate the accuracy of the detected smell type.
STEP 3: Evaluate the precision of the location identified.
STEP 4: Assign a score based on the rating rubric.
STEP 5: Calculate overall precision and recall metrics.

# Input Data
## Ground Truth Smells
{ground_truth}

## Detected Smells
{detected_smells}

{format_instructions}
"""

# 3. Create the evaluation parser and prompt
eval_parser = PydanticOutputParser(pydantic_object=CodeSmellEvaluationResult)
eval_prompt = PromptTemplate(
    template=eval_template,
    input_variables=["ground_truth", "detected_smells"],
    partial_variables={"format_instructions": eval_parser.get_format_instructions()}
)

# 4. Create the evaluation function
def evaluate_smell_detection(ground_truth_data, detected_smells, llm):
    """
    Evaluate the quality of code smell detection by comparing with ground truth.
    
    Args:
        ground_truth_data: Dictionary with ground truth smells
        detected_smells: Dictionary with detected smells
        llm: LangChain LLM instance
    
    Returns:
        Evaluation results with scores and metrics
    """
    # Convert data to JSON strings
    ground_truth_str = json.dumps(ground_truth_data, indent=2)
    detected_str = json.dumps(detected_smells, indent=2)
    
    # Format the prompt
    formatted_prompt = eval_prompt.format(
        ground_truth=ground_truth_str,
        detected_smells=detected_str
    )
    
    # Get response from LLM
    try:
        output = llm.invoke(formatted_prompt)
        parsed_output = eval_parser.parse(output.content)
        return parsed_output
    except Exception as e:
        print(f"Error during evaluation: {e}")
        print(f"Raw output: {output.content if 'output' in locals() else 'No output'}")
        return None

# 5. Helper function to create ground truth dataset
def create_ground_truth(file_path, manual_annotations):
    """Create ground truth data structure"""
    return {
        "file_path": file_path,
        "smells": manual_annotations
    }

# 6. Main evaluation function
def evaluate_code_analysis(file_path, manual_annotations, llm):
    """Run a full evaluation on a code file"""
    # Get the code content
    with open(file_path, 'r') as f:
        code_content = f.read()
    
    # Run code smell detection
    analysis_result = analyze_code_with_structure(code_content)
    
    if not analysis_result:
        print("Failed to analyze code")
        return None
    
    # Format detected smells
    detected_smells = {
        "file_path": file_path,
        "smells": []
    }
    
    for smell in analysis_result.smells_detected:
        detected_smells["smells"].append({
            "smell_type": smell.smell_type,
            "location": smell.location,
            "description": smell.description,
            "severity": str(smell.severity),
            "refactoring": smell.refactoring_suggestion
        })
    
    # Create ground truth
    ground_truth = create_ground_truth(file_path, manual_annotations)
    
    # Run evaluation
    evaluation = evaluate_smell_detection(ground_truth, detected_smells, llm)
    
    # Display results
    if evaluation:
        print(f"Overall Score: {evaluation.overall_score:.2f}/5.0")
        print(f"Precision: {evaluation.precision:.2f}")
        print(f"Recall: {evaluation.recall:.2f}")
        print(f"\nSummary: {evaluation.summary}\n")
        
        print("Individual Evaluations:")
        for i, eval_item in enumerate(evaluation.evaluations, 1):
            print(f"\n{i}. {eval_item.detected_smell} ({eval_item.location})")
            print(f"   Score: {eval_item.score.value}")
            print(f"   Matched with: {eval_item.ground_truth_match if eval_item.ground_truth_match else 'No match'}")
            print(f"   Justification: {eval_item.justification}")
    
    return evaluation

In [ ]:
# Example ground truth data for a file - here we can utilize data from static analyzers such as Sonarqube etc.
evaluation_files = {
    "pixel-dungeon/src/com/watabou/pixeldungeon/levels/HallsLevel.java": [
        {
            "smell_type": "MAGIC_NUMBER",
            "location": "multiple locations: lines 25-26, 37, 52-53, 85, 89",
            "severity": "MEDIUM"
        },
        {
            "smell_type": "LONG_METHOD",
            "location": "decorate() method, lines 74-101",
            "severity": "MEDIUM"
        },
        {
            "smell_type": "CONDITIONAL_COMPLEXITY",
            "location": "decorate() method, lines 76-89",
            "severity": "LOW"
        },
        {
            "smell_type": "DUPLICATED_CODE",
            "location": "tileName() and tileDesc() methods, lines 104-138",
            "severity": "LOW"
        },
        {
            "smell_type": "DEAD_CODE",
            "location": "map[i] == 63 condition in addVisuals method, line 151",
            "severity": "MEDIUM"
        },
        {
            "smell_type": "FEATURE_ENVY",
            "location": "addVisuals method, lines 149-153",
            "severity": "LOW"
        },
        {
            "smell_type": "PRIMITIVE_OBSESSION",
            "location": "throughout class, using boolean arrays and ints for terrain",
            "severity": "LOW"
        }
    ]
}

# Run evaluation on a single file
java_file = "pixel-dungeon/src/com/watabou/pixeldungeon/levels/HallsLevel.java"
evaluation = evaluate_code_analysis(java_file, evaluation_files, llm)

KeyboardInterrupt: 

In [ ]:
# Run evaluations and collect results
results = {}
for file_path, annotations in evaluation_files.items():
    print(f"\n=== Evaluating {file_path} ===")
    eval_result = evaluate_code_analysis(file_path, annotations, llm)
    if eval_result:
        results[file_path] = eval_result

# Calculate overall metrics
if results:
    total_score = sum(result.overall_score for result in results.values())
    avg_score = total_score / len(results)
    avg_precision = sum(result.precision for result in results.values()) / len(results)
    avg_recall = sum(result.recall for result in results.values()) / len(results)

    print("\n=== OVERALL EVALUATION ===")
    print(f"Average Score: {avg_score:.2f}/5.0")
    print(f"Average Precision: {avg_precision:.2f}")
    print(f"Average Recall: {avg_recall:.2f}")


=== Evaluating pixel-dungeon/src/com/watabou/pixeldungeon/levels/HallsLevel.java ===


In [ ]:
import requests
import json

# Configuration
sonar_url = "http://localhost:9000"  # SonarQube server URL
#sonar_token = "squ_22e01b75153105a4fa214640736e4714f8f71254"
#project_key = "pixeldungeon"
sonar_token = "squ_2fcc7bef115621213484482b299e2a029a71c3aa"
project_key = "arthas"

# API endpoint for issues search
url = f"{sonar_url}/api/issues/search"

# Parameters to get only code smells
params = {
    "componentKeys": project_key,
    "types": "CODE_SMELL",
    "ps": 5  # page size (max 500)
}

# Authentication with token
auth = (sonar_token, "")

# Make the API request
response = requests.get(url, params=params, auth=auth)
code_smells = response.json()

# Process results
print(json.dumps(code_smells, indent=2))  
# Handle pagination if needed (total > 500)
total = code_smells.get("total", 0)
if total > 500:
    print(f"Found {total} issues, need to handle pagination")


{
  "total": 1978,
  "p": 1,
  "ps": 5,
  "paging": {
    "pageIndex": 1,
    "pageSize": 5,
    "total": 1978
  },
  "effortTotal": 14806,
  "issues": [
    {
      "key": "aa865edb-1730-412b-9a9d-2f884713fac7",
      "rule": "java:S3776",
      "severity": "CRITICAL",
      "component": "arthas:core/src/main/java/com/taobao/arthas/core/command/klass100/ClassLoaderCommand.java",
      "project": "arthas",
      "line": 406,
      "hash": "49b3709adf3e9f83dd649a2b925196a7",
      "textRange": {
        "startLine": 406,
        "endLine": 406,
        "startOffset": 51,
        "endOffset": 59
      },
      "flows": [
        {
          "locations": [
            {
              "component": "arthas:core/src/main/java/com/taobao/arthas/core/command/klass100/ClassLoaderCommand.java",
              "textRange": {
                "startLine": 409,
                "endLine": 409,
                "startOffset": 8,
                "endOffset": 11
              },
              "msg": "+1",

In [ ]:
import pandas as pd
import mysql.connector

# Connect to MySQL
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="dacos"
)

# Create cursor and run queries
cursor = conn.cursor()
cursor.execute("SELECT id, designite_id, has_smell, is_class, path_to_file, project_name, sample_constraints, smells FROM tagman5.sample WHERE smells IN (2) LIMIT 10")
rows = cursor.fetchall()

column_names = [column[0] for column in cursor.description]

df = pd.DataFrame(rows, columns=column_names)

print(f"Retrieved {len(df)} samples with smells")
display(df)

if len(df) > 0:
    print("\nAccessing first row:")
    print(f"File path: {df.iloc[0]['path_to_file']}")
    print(f"Project name: {df.iloc[0]['project_name']}")
    print(f"Has smell: {'Yes' if df.iloc[0]['has_smell'] else 'No'}")

conn.close()

Retrieved 10 samples with smells


,id,designite_id,has_smell,is_class,path_to_file,project_name,sample_constraints,smells
0,2386,538,1,0,/codesplit_java_method/Blankj_AndroidUtilCode/...,Blankj_AndroidUtilCode,5,2
1,2387,542,1,0,/codesplit_java_method/Blankj_AndroidUtilCode/...,Blankj_AndroidUtilCode,6,2
2,2406,532,1,0,/codesplit_java_method/Blankj_AndroidUtilCode/...,Blankj_AndroidUtilCode,9,2
3,2407,536,1,0,/codesplit_java_method/Blankj_AndroidUtilCode/...,Blankj_AndroidUtilCode,7,2
4,2494,552,1,0,/codesplit_java_method/Blankj_AndroidUtilCode/...,Blankj_AndroidUtilCode,3,2
5,2501,5129,1,0,/codesplit_java_method/Blankj_AndroidUtilCode/...,Blankj_AndroidUtilCode,3,2
6,2551,5487,1,0,/codesplit_java_method/Blankj_AndroidUtilCode/...,Blankj_AndroidUtilCode,3,2
7,2567,5480,1,0,/codesplit_java_method/Blankj_AndroidUtilCode/...,Blankj_AndroidUtilCode,3,2
8,2569,5479,1,0,/codesplit_java_method/Blankj_AndroidUtilCode/...,Blankj_AndroidUtilCode,3,2
9,2577,5441,1,0,/codesplit_java_method/Blankj_AndroidUtilCode/...,Blankj_AndroidUtilCode,3,2



Accessing first row:
File path: /codesplit_java_method/Blankj_AndroidUtilCode/Blankj_AndroidUtilCode/com.blankj.subutil.util/ContentProvider4SubUtil/query.code
Project name: Blankj_AndroidUtilCode
Has smell: Yes


In [27]:
import os
import pandas as pd
import mysql.connector

# Connect to MySQL
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="dacos"
)

# Create cursor and run queries
cursor = conn.cursor()
cursor.execute("SELECT id, designite_id, has_smell, is_class, path_to_file, project_name, sample_constraints, smells FROM tagman5.sample" \
" WHERE smells IN (2) AND project_name='alibaba_arthas' LIMIT 10")
rows = cursor.fetchall()

column_names = [column[0] for column in cursor.description]

df = pd.DataFrame(rows, columns=column_names)

print(f"Retrieved {len(df)} samples with smells")
display(df)

if len(df) > 0:
    print("\nAccessing first row:")
    file_path = df.iloc[0]['path_to_file']
    print(f"File path: {file_path}")
    print(f"Project name: {df.iloc[0]['project_name']}")
    print(f"Has smell: {'Yes' if df.iloc[0]['has_smell'] else 'No'}")
    
    # Construct new file path with .java extension
    dir_name, file_name = os.path.split(file_path)
    print(dir_name, file_name)
    base_name = os.path.splitext(file_name)[0]
    new_file_path = os.path.join(dir_name, base_name + ".java")
    
    print(f"\nNew file path (with .java): {new_file_path}")
    
    # Analyze the code using the function defined in previous cells
    try:
        # First check if the file exists
        if os.path.exists(new_file_path):
            # Use display_code_analysis which is already defined
            print(f"\nAnalyzing code in: {new_file_path}")
            display_code_analysis(new_file_path)
        else:
            print(f"\nFile not found: {new_file_path}")
            print("Please check if the path is correct.")
    except Exception as e:
        print(f"\nError analyzing file: {e}")
        
conn.close()

Retrieved 10 samples with smells


,id,designite_id,has_smell,is_class,path_to_file,project_name,sample_constraints,smells
0,44635,59787,1,0,/codesplit_java_method/alibaba_arthas/alibaba_...,alibaba_arthas,2,2
1,44787,63642,1,0,/codesplit_java_method/alibaba_arthas/alibaba_...,alibaba_arthas,2,2
2,44917,59836,1,0,/codesplit_java_method/alibaba_arthas/alibaba_...,alibaba_arthas,2,2
3,44951,59902,1,0,/codesplit_java_method/alibaba_arthas/alibaba_...,alibaba_arthas,2,2
4,44998,59960,1,0,/codesplit_java_method/alibaba_arthas/alibaba_...,alibaba_arthas,2,2
5,45029,59926,1,0,/codesplit_java_method/alibaba_arthas/alibaba_...,alibaba_arthas,2,2
6,45044,59981,1,0,/codesplit_java_method/alibaba_arthas/alibaba_...,alibaba_arthas,2,2
7,45061,59814,1,0,/codesplit_java_method/alibaba_arthas/alibaba_...,alibaba_arthas,2,2
8,45136,59195,1,0,/codesplit_java_method/alibaba_arthas/alibaba_...,alibaba_arthas,2,2
9,45155,59172,1,0,/codesplit_java_method/alibaba_arthas/alibaba_...,alibaba_arthas,2,2



Accessing first row:
File path: /codesplit_java_method/alibaba_arthas/alibaba_arthas/com.taobao.arthas.core.command.express/ArthasObjectPropertyAccessor/setPossibleProperty.code
Project name: alibaba_arthas
Has smell: Yes
/codesplit_java_method/alibaba_arthas/alibaba_arthas/com.taobao.arthas.core.command.express/ArthasObjectPropertyAccessor setPossibleProperty.code

New file path (with .java): /codesplit_java_method/alibaba_arthas/alibaba_arthas/com.taobao.arthas.core.command.express/ArthasObjectPropertyAccessor/setPossibleProperty.java

File not found: /codesplit_java_method/alibaba_arthas/alibaba_arthas/com.taobao.arthas.core.command.express/ArthasObjectPropertyAccessor/setPossibleProperty.java
Please check if the path is correct.
